In [49]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [50]:
df_org = pd.read_csv("final_scout_not_dummy.csv")

In [51]:
df_org.head(1)

,make_model,body_type,price,vat,km,Type,Fuel,Gears,Comfort_Convenience,Entertainment_Media,Extras,Safety_Security,age,Previous_Owners,hp_kW,Inspection_new,Paint_Type,Upholstery_type,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb
0,Audi A1,Sedans,15770,VAT deductible,56013.0,Used,Diesel,7.0,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",3.0,2.0,66.0,1,Metallic,Cloth,Automatic,1422.0,1220.0,front,3.8


In [52]:
df = pd.concat([df_org["make_model"],df_org["body_type"], df_org["km"], df_org["age"], df_org["Gearing_Type"],df_org["Fuel"],df_org["hp_kW"], df_org["price"]], axis=1)
df

,make_model,body_type,km,age,Gearing_Type,Fuel,hp_kW,price
0,Audi A1,Sedans,56013.000000,3.0,Automatic,Diesel,66.0,15770
1,Audi A1,Sedans,80000.000000,2.0,Automatic,Benzine,141.0,14500
2,Audi A1,Sedans,83450.000000,3.0,Automatic,Diesel,85.0,14640
3,Audi A1,Sedans,73000.000000,3.0,Automatic,Diesel,66.0,14500
4,Audi A1,Sedans,16200.000000,3.0,Automatic,Diesel,66.0,16790
...,...,...,...,...,...,...,...,...
15910,Renault Espace,Van,1647.362609,0.0,Automatic,Diesel,147.0,39950
15911,Renault Espace,Van,9900.000000,0.0,Automatic,Benzine,165.0,39885
15912,Renault Espace,Van,15.000000,0.0,Automatic,Diesel,146.0,39875
15913,Renault Espace,Van,10.000000,0.0,Automatic,Diesel,147.0,39700


In [53]:
df.to_csv("final_model.csv", index=False)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15915 entries, 0 to 15914
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   make_model    15915 non-null  object 
 1   body_type     15915 non-null  object 
 2   km            15915 non-null  float64
 3   age           15915 non-null  float64
 4   Gearing_Type  15915 non-null  object 
 5   Fuel          15915 non-null  object 
 6   hp_kW         15915 non-null  float64
 7   price         15915 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 994.8+ KB


# Train test split

In [55]:
X=df.drop("price", axis=1)
y=df.price

In [56]:
cat = X.select_dtypes("object").columns
cat

Index(['make_model', 'body_type', 'Gearing_Type', 'Fuel'], dtype='object')

In [57]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X[cat] = enc.fit_transform(X[cat])
X.head()

,make_model,body_type,km,age,Gearing_Type,Fuel,hp_kW
0,0.0,4.0,56013.0,3.0,0.0,1.0,66.0
1,0.0,4.0,80000.0,2.0,0.0,0.0,141.0
2,0.0,4.0,83450.0,3.0,0.0,1.0,85.0
3,0.0,4.0,73000.0,3.0,0.0,1.0,66.0
4,0.0,4.0,16200.0,3.0,0.0,1.0,66.0


In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

print("Train features shape : ", X_train.shape)
print("Train target shape   : ", y_train.shape)
print("Test features shape  : ", X_test.shape)
print("Test target shape    : ", y_test.shape)

Train features shape :  (12732, 7)
Train target shape   :  (12732,)
Test features shape  :  (3183, 7)
Test target shape    :  (3183,)


# Modeling

In [59]:
import xgboost

In [60]:
from xgboost import XGBRegressor

In [61]:
xgb_model = XGBRegressor(random_state=101, objective="reg:squarederror")

In [62]:
xgb_model.fit(X_train, y_train)

XGBRegressor(objective='reg:squarederror', random_state=101)

In [63]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [64]:
def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [65]:
train_val(xgb_model, X_train, y_train, X_test, y_test)

,train,test
R2,9.174693e-01,9.186673e-01
mae,1.378360e+03,1.352930e+03
mse,4.559036e+06,4.180628e+06
rmse,2.135190e+03,2.044658e+03


# Final Model and Model Deployment

In [45]:
import pickle

In [46]:
final_model = XGBRegressor(objective="reg:squarederror").fit(X_train,y_train)

In [47]:
pickle.dump(final_model, open("auto_scout.pkl", 'wb'))

In [48]:
pickle.dump(enc, open("autoscout_encoder.pkl", 'wb'))